(multilevel-sm-notebook)=
# Modelli di Equazioni Strutturali Multilivello

In [18]:
source("_common.R")
suppressPackageStartupMessages({
    library("lavaan")
    library("semPlot")
    library("knitr")
    library("markdown")
    library("patchwork")
    library("lme4")
    library("here")
})

set.seed(42)

I dati prodotti dalla maggior parte delle ricerche nelle scienze sociali e nella sanità pubblica presentano strutture multi-livello, in cui i dati sono raggruppati in cluster e le osservazioni all'interno di ciascun cluster sono correlate. Nella maggior parte dei casi, se non in tutti, il modello classico di equazioni strutturali non è adatto per l'analisi di dati multi-livello o raggruppati per produrre stime valide ed efficienti, a causa delle correlazioni presenti nei dati su più livelli. Pertanto, la modellazione di equazioni strutturali dovrebbe essere estesa per incorporare la struttura dei dati multi-livello.

In questo capitolo introdurremo l'implementazione in `lavaan` per l'analisi SEM multilivello. Utilizzeremo un set di dati artificiali ricavato dal sito di MPlus.

In [2]:
dat <- read.table("http://statmodel.com/usersguide/chap9/ex9.6.dat")
names(dat) <- c("y1", "y2", "y3", "y4", "x1", "x2", "w", "clus")
head(dat)

,y1,y2,y3,y4,x1,x2,w,clus
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,2.203250,1.858861,1.738477,2.244863,1.142800,-0.796987,-0.149501,1
2,1.934917,2.127876,0.083120,2.509436,1.949033,-0.122764,-0.149501,1
3,0.321955,0.977231,-0.835405,0.558367,-0.716481,-0.767064,-0.149501,1
4,0.073154,-1.743092,-2.310271,-1.514332,-2.649131,0.637570,-0.149501,1
5,-1.214906,0.452618,0.372610,-1.790372,-0.262916,0.302564,-0.149501,1
6,0.298330,-1.820272,0.561335,-2.090582,-0.944963,1.363045,0.319335,2


Il data frame è costituito da 1000 righe:

In [4]:
dim(dat)

[1] 1000    8

Ci sono soltanto 110 cluster (`clus`), il che significa che ci sono misure ripetute per ciascun cluster (possiamo immaginare che i cluster corrispondano ai soggetti):

In [3]:
length(unique(dat$clus))

[1] 110

Analizzeremo questi dati mediante un modello di equazioni strutturali multilivello (SEM). Iniziamo a definire il modello SEM appropriato per questi dati.

In [ ]:
model <- "
    level: 1
        fw =~ y1 + y2 + y3 + y4
        fw ~ x1 + x2

    level: 2
        fb =~ y1 + y2 + y3 + y4

    # optional
    y1 ~~ 0*y1
    y2 ~~ 0*y2
    y3 ~~ 0*y3
    y4 ~~ 0*y4
    fb ~ w
"

### Livelli nel Modello SEM Multilivello

1. **Livello 1:** Il primo livello rappresenta le misurazioni individuali. Nel modello, `fw` è un fattore latente che viene definito dalle quattro variabili osservate `y1`, `y2`, `y3`, `y4`. Queste variabili sono influenzate da due predittori `x1` e `x2`. Questo livello cattura la variazione a livello individuale.

2. **Livello 2:** Il secondo livello considera le differenze tra i gruppi o i cluster (indicati dalla variabile `clus`). Qui, `fb` è un altro fattore latente definito dalle stesse variabili osservate `y1`, `y2`, `y3`, `y4`. La variabile `w` agisce come un predittore a livello di cluster.

### Coefficiente di Correlazione Intraclass (ICC)

Lo scopo dell'analisi statistica è quello di calcolare il Coefficiente di Correlazione Intraclasse (ICC) che è una misura utilizzata per quantificare la somiglianza delle misurazioni all'interno dei gruppi o cluster. In un contesto multilivello, l'ICC indica quanto della varianza totale è attribuibile alle differenze tra i gruppi.

#### ICC in SEM Multilivello

Nel modello SEM, l'ICC può essere calcolato per ogni variabile osservata (`y1`, `y2`, `y3`, `y4`) come la proporzione della varianza tra i cluster rispetto alla varianza totale (sia tra che all'interno dei cluster). Questo aiuta a capire quanto la variazione in queste variabili sia influenzata dal gruppo di appartenenza (`clus`).

#### Calcolo dell'ICC con `lmer`

Per iniziare, poniamoci il problema di calcolare ICC mediante un modello lineare multilivello. Svolgeremo questi calcoli con `lmer`. Il modello `lmer` considera ogni variabile osservata separatamente, fornendo un'analisi indipendente per ciascuna. Per `y1`, per esempio, abbiamo:

In [19]:
model_lmer <- lmer(y1 ~ 1 + (1 | clus), data = dat)

Calcoliamo l'ICC:

In [20]:
 varianza_cluster <- VarCorr(model_lmer)$clus[1]
 varianza_residua <- attr(VarCorr(model_lmer), "sc")^2
 ICC <- varianza_cluster / (varianza_cluster + varianza_residua)
 ICC

[1] 0.1295362

Questo valore di ICC, pari a 0.129, indica che solo il 12.9% della variazione totale nelle variabili osservate può essere attribuito alle differenze tra gli studenti (considerati come cluster individuali). In altre parole, una modesta parte della variazione complessiva nei dati è spiegata dalle differenze individuali tra gli studenti. Un ICC relativamente basso, come in questo caso, suggerisce che la maggior parte della variazione nei dati non è fortemente influenzata dalle caratteristiche o dai comportamenti individuali degli studenti, ma potrebbe essere dovuta ad altri fattori. In un contesto educativo, ciò potrebbe indicare che fattori esterni agli studenti stessi, come l'ambiente scolastico, le metodologie didattiche, o le caratteristiche del programma di studi, hanno un impatto maggiore sulla variazione osservata rispetto alle differenze individuali tra gli studenti.

Calcoliamo ora l'ICC con un modello SEM multilivello.

In [21]:
 fit <- sem(model,
     data = dat,
     cluster = "clus",
     fixed.x = FALSE
 )

In [14]:
summary(fit) |>
    print()

lavaan 0.6.16 ended normally after 32 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        26

  Number of observations                          1000
  Number of clusters [clus]                        110

Model Test User Model:
                                                      
  Test statistic                                 0.000
  Degrees of freedom                                17
  P-value (Chi-square)                           1.000

Parameter Estimates:

  Standard errors                             Standard
  Information                                 Observed
  Observed information based on                Hessian


Level 1 [within]:

Latent Variables:
                   Estimate  Std.Err  z-value  P(>|z|)
  fw =~                                               
    y1                1.000                           
    y2                0.987    0.034  

In [13]:
fitMeasures(fit) |>
    print()

                 npar                  fmin                 chisq 
               26.000                 3.982                 0.000 
                   df                pvalue        baseline.chisq 
               17.000                 1.000              3075.708 
          baseline.df       baseline.pvalue                   cfi 
               24.000                 0.000                 1.000 
                  tli                  nnfi                   rfi 
                1.008                 1.008                 1.000 
                  nfi                  pnfi                   ifi 
                1.000                 0.708                 1.006 
                  rni                  logl     unrestricted.logl 
                1.006             -9596.778             -9629.425 
                  aic                   bic                ntotal 
            19245.556             19373.158              1000.000 
                 bic2                 rmsea        rmsea.ci.lo

Quando calcoliamo l'ICC con lavaan, ottieniamo valori separati per ciascuna variabile osservata nel contesto del modello multilivello. Questi valori riflettono quanto la varianza in ogni variabile sia attribuibile alle differenze tra i gruppi, ma all'interno del framework più ampio delle relazioni specificate nel modello SEM. L'ICC per ciascuno dei 4 item si trova nel modo seguente:

In [17]:
lavInspect(fit, "icc") |>
    print()

   y1    y2    y3    y4    x1    x2 
0.182 0.193 0.170 0.185 0.000 0.000 


Nel caso di `y1`, le stime fornite da `lmer` sono simili a quelle fornite dal modello SEM multilivello.

## Considerazioni Conclusive

In questo capitolo, abbiamo illustrato il modello di equazioni strutturali multilivello utilizzando `lavaan`. Come evidenziato dall'esempio, l'implementazione in `lavaan` è molto diretta, richiedendo solo l'inclusione dell'opzione `cluster` nella funzione `sem`. È importante sottolineare che, al momento, `lavaan` supporta solo modelli SEM a due livelli. 

Nell'ambito dei modelli SEM multilivello, abbiamo visto come l'interpretazione dei coefficienti di correlazione intra-classe (ICC) possa fornire intuizioni significative sulla variazione dei dati all'interno di gruppi o cluster. Un ICC basso, come quello osservato nell'esempio (0.129), indica che una porzione minore della variazione totale è attribuibile alle differenze tra i cluster. Nel contesto specifico dei nostri dati, dove ogni studente è considerato un cluster individuale, ciò suggerisce che fattori esterni agli studenti stessi potrebbero giocare un ruolo più significativo nella variazione osservata rispetto alle caratteristiche individuali degli studenti.

In conclusione, la modellazione di equazioni strutturali multilivello è uno strumento potente e flessibile nell'analisi di dati strutturati gerarchicamente. `lavaan`, sebbene limitato ai modelli a due livelli, fornisce un approccio accessibile e diretto per questi tipi di analisi. Per modelli più complessi e a più livelli, Mplus e Stata offrono soluzioni alternative che possono gestire una gamma più ampia di esigenze analitiche. 